## Scrapy之settings详解
　　参考：[Scrapy爬虫入门教程Settings（设置）](https://www.jianshu.com/p/df9c0d1e9087)<br>
### 一、Settings说明
　　Scrapy设置允许您自定义所有Scrapy组件的行为，包括核心，扩展，管道和爬虫本身。设置的基础结构提供了键值映射的全局命名空间，代码可以使用它从中提取配置值。并可以通过不同的机制来填充设置。

#### 1、指定设置
　　使用Scrapy必须要告诉它使用哪些设置。可以使用环境变量SCRAPY_SETTINGS_MODULE来进行制定，而且SCRAPY_SETTINGS_MODULE需要在Python的路径语法中，例如myproject.settings。

#### 2、配置设置
　　可以使用不同类型的机制来填充设置，每个机制具有不同的优先级。这里是按优先级降序排列的列表：
>1、命令行选项（最高优先级）
2、每个Spider的Settings<br>
3、项目的Settings<br>
4、每个命令的默认Settings<br>
5、默认的全局Settings（最低优先级）<br>

##### 1.命令行选项
　　在使用命令行时提供的参数拥有最高的优先级，将会覆盖所有其他方式设置的相同Settings选项。你可以使用-s(或者--set)来明确指定覆盖一个或多个Settings。如下：
>scrapy crawl myspider -s LOG_FILE=scrapy.log
##### 2.每个Spider的Settings
　　在每个Spider中，是可以定义这个Spider所特有的Settings的。只需要在Spider类中，定义好custom_settings这个类属性即可。如下：
>class MySpider(scrapy.Spider):<br>
    name = 'myspider'<br><br>
    custom_settings = {<br>
        'SOME_SETTING': 'some value',<br>
    }
##### 3.项目的Settings模块
　　项目的Settings是Scrapy项目的标准配置文件，我们大部分的设置都会在这个地方配置好。对于一个标准的Scrapy项目来说，这意味着我们在settings.py这个文件中添加或者修改配置的字段。<br>
　　这个文件基本的设置都以注释的形式保存在文件中。<br>

##### 4.每个命令的默认Settings
　　每一个Scrapy命令都有各自的默认Settings，会覆盖掉默认的全局设置。这些Settings设定在命令类的default_settings属性中。

##### 5.默认全局设置
　　默认的全局变量设定在scrapy.settings.default_settings模块中。

### 二、访问Settings
　　在Scrapy中，我们常常需要去获取Settings中的某一个值来进行使用，或者是将我们的自定义Settings写到settings.py文件中，然后在代码中进行获取。Scrapy提供了一些非常方便的钩子和方法来获得Settings。

1.Spider中获取Settings
在Spider中获取Settings非常的方便，直接访问self.settings即可：

>class MySpider(scrapy.Spider):<br>
    name = 'myspider'<br>
    start_urls = ['http://example.com']<br><br>
    def parse(self, response):<br>
        print("Existing settings: %s" % self.settings.attributes.keys())<br>
        
　　注意：settings属性是在基础的Spider类中的__init__()中设定的，这意味着如果你想要在__init__()中使用settings的话，那么你需要重写from_crawler()方法。

2.extensions、middlewares、item pipelines中获取Settings
　　在extensions，middlewares和item pipelines中想要获取Settings的话，那么就只能使用from_crawler()方法。from_crawler()是一个类方法，这个方法一般用来生成当前这个组件的实例，在调用的时候会传入crawler参数，通过这个crawler参数为桥梁，我们就可以获取Settings了：
>class MyExtension(object):<br>
    def __init__(self, log_is_enabled=False):<br>
        if log_is_enabled:<br>
            print("log is enabled!")<br><br> 
    @classmethod<br>
    def from_crawler(cls, crawler):<br>
        settings = crawler.settings<br>
        return cls(settings.getbool('LOG_ENABLED'))<br>
        
　　同样要注意的是，由于from_crawler()是用来生成组件的实例的，所以最终需要return一个类的实例，然后把我们要获得的Settings当作参数传递到这个实例中去，最后再在__init__()方法中进行接收。

### 三、内置基础设置（Built-in settings）
　　Scrapy中定义了许多基础的默认全局Settings，以下是一些常见设置的默认值和作用范围。这些设置的默认范围指的是只有启用特定的组件，那么这条设置才会生效。

#### 1、常用设置
##### 1、BOT_NAME
　　默认值：'scrapybot'，这个Scrapy项目的Bot名称。
　　BOT_NAME将用于在默认状态下构造User-Agent，也用于日志记录。当使用startproject命令创建项目时，将会自动使用项目名称来填充这个设置。

##### 2、DEFAULT_REQUEST_HEADERS
　　Scrapy的Request中默认的请求头，将会在DefaultHeadersMiddleware中被填充。
>{<br>
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',<br>
    'Accept-Language': 'en',<br>
}<br>
##### 3、DOWNLOAD_TIMEOUT
　　默认值：180<br>
　　Downloader下载的最大超时时间。这个设置可以在每个Spider中使用download_timeout属性进行设置，也可以在每个Request中可以使用Request.meta中的download_timeout字段进行设置。<br>

##### 4、ROBOTSTXT_OBEY
　　默认值：True<br>
　　作用域：scrapy.downloadermiddlewares.robotstxt<br>
　　如果设置为True，那么Scrapy将会自动遵守robots.txt协议。<br>

##### 5、COOKIES_ENABLED
　　默认值：True<br>
　　是否开启cookies middleware。如果关闭这个设置的话，请求时将不会发送任何cookies到网站。<br>

#### 2、并发控制
##### 1、CONCURRENT_ITEMS
　　默认值：100<br>
在Item的处理器中并发处理的最大Item数量。<br>
##### 2、CONCURRENT_REQUESTS
　　默认值：16<br>
　　Downloader同时处理Request的最大数量<br>
##### 3、CONCURRENT_REQUESTS_PER_DOMAIN
　　默认值：8<br>
　　任何单个域名同时发起的最大Request数量。用来控制单个域名的并发量。<br>
##### 4、CONCURRENT_REQUESTS_PER_IP
　　默认值：0<br>
　　任何单个IP同时发起的最大Request数量。如果这个值不为0的话，那么CONCURRENT_REQUESTS_PER_DOMAIN将会被忽略。也就是说，并发量的限制条件，要么只能使用IP，要么只能使用Domain。<br>
　　这一个设置同样会影响DOWNLOAD_DELAY，使其判断标准变为IP。<br>
##### 5、DOWNLOAD_DELAY
　　默认值：0<br>
　　Downloader从同一网站上访问页面应该等待的时间。这个设置常常被用来控制爬取的速度，以免对网站造成太大的压力。这个值的单位为秒，并且可以使用小数。<br>
　　这个值同样被RANDOMIZE_DOWNLOAD_DELAY所影响。在默认的情况下，Scrapy不会每次都等待一个固定的时间，而是使用一个范围在0.5-1.5之间的值乘以DOWNLOAD_DELAY。<br>
　　当CONCURRENT_REQUESTS_PER_IP的值不为0时，延迟将按照每个IP判断，而不再是Domain。<br>
　　这个设置可以在Spider中通过download_delay属性进行设置。

##### 6、RANDOMIZE_DOWNLOAD_DELAY
　　默认值：True<br>
　　如果启用，那么DOWNLOAD_DELAY等待的时间将会随机乘以范围在0.5-1.5之间的一个小数。这个设置的作用是使爬虫的等待时间更有随机性，更难以被检测。

#### 3、组件启用与设置<br>
##### 1、DOWNLOADER_MIDDLEWARES
　　默认值：{}<br>
##### 2、DOWNLOADER_MIDDLEWARES_BASE
　　默认值：<br>
>{<br>
    'scrapy.downloadermiddlewares.robotstxt.RobotsTxtMiddleware': 100,<br>
    'scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware': 300,<br>
    'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware': 350,<br>
    'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware': 400,<br>
    'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware': 500,<br>
    'scrapy.downloadermiddlewares.retry.RetryMiddleware': 550,<br>
    'scrapy.downloadermiddlewares.ajaxcrawl.AjaxCrawlMiddleware': 560,<br>
    'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware': 580,<br>
    'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware': 590,<br>
    'scrapy.downloadermiddlewares.redirect.RedirectMiddleware': 600,<br>
    'scrapy.downloadermiddlewares.cookies.CookiesMiddleware': 700,<br>
    'scrapy.downloadermiddlewares.httpproxy.HttpProxyMiddleware': 750,<br>
    'scrapy.downloadermiddlewares.stats.DownloaderStats': 850,<br>
    'scrapy.downloadermiddlewares.httpcache.HttpCacheMiddleware': 900,<br>
}
##### 3、EXTENSIONS
　　默认值：{}<br>
　　包含项目中启用的扩展名及其顺序的字典。<br>

##### 4、EXTENSIONS_BASE
　　默认值：<br>
>{<br>
    'scrapy.extensions.corestats.CoreStats': 0,<br>
    'scrapy.extensions.telnet.TelnetConsole': 0,<br>
    'scrapy.extensions.memusage.MemoryUsage': 0,<br>
    'scrapy.extensions.memdebug.MemoryDebugger': 0,<br>
    'scrapy.extensions.closespider.CloseSpider': 0,<br>
    'scrapy.extensions.feedexport.FeedExporter': 0,<br>
    'scrapy.extensions.logstats.LogStats': 0,<br>
    'scrapy.extensions.spiderstate.SpiderState': 0,<br>
    'scrapy.extensions.throttle.AutoThrottle': 0,<br>
}
##### 5、ITEM_PIPELINES
　　默认值：{}

##### 6、ITEM_PIPELINES_BASE
　　默认值：{}

##### 7、SPIDER_MIDDLEWARES
　　默认值：{}

##### 8、SPIDER_MIDDLEWARES_BASE
　　默认值：
>{<br>
    'scrapy.spidermiddlewares.httperror.HttpErrorMiddleware': 50,<br>
    'scrapy.spidermiddlewares.offsite.OffsiteMiddleware': 500,<br>
    'scrapy.spidermiddlewares.referer.RefererMiddleware': 700,<br>
    'scrapy.spidermiddlewares.urllength.UrlLengthMiddleware': 800,<br>
    'scrapy.spidermiddlewares.depth.DepthMiddleware': 900,<br>
}

#### 4、记录日志
##### 1、LOG_ENABLED
　　默认值：True<br>
　　是否开启日志记录。<br>

##### 2、LOG_ENCODING
　　默认值：'utf-8'<br>
　　记录日志使用的编码。<br>

##### 3、LOG_FILE
　　默认值：None<br>
　　日志输出的文件路径。<br>

##### 4、LOG_FORMAT
　　默认值：'%(asctime)s [%(name)s] %(levelname)s: %(message)s'<br>
　　日志记录的格式。<br>

##### 5、LOG_DATEFORMAT
　　默认值：'%Y-%m-%d %H:%M:%S'<br>
　　记录日志的时间格式。<br>

##### 6、LOG_LEVEL
　　默认值：'DEBUG'<br>
　　输出日志的最小级别，可用的级别有：CRITICAL, ERROR, WARNING, INFO, DEBUG。<br>

##### 7、LOG_STDOUT
　　默认值：False<br>
　　如果设置为True，将会把所有的标准输出重定向到日志中。例如print打印的信息和异常信息。<br>

#### 5、内存控制<br>
#####  1、MEMDEBUG_ENABLED
　　默认值：False<br>
　　是否开启内存调试。<br>

##### 2、MEMDEBUG_NOTIFY
　　默认值：[]<br>
　　当开启内存调试的时候，可以在这个列表中设置邮箱地址，那么内存调试的报告将会发送到邮箱中。如：<br>

##### 3、MEMDEBUG_NOTIFY = ['user@example.com']
##### 4、MEMUSAGE_ENABLED
　　默认值：True<br>
　　作用域：scrapy.extensions.memusage<br>
　　是否开启内存使用的扩展。这个扩展会一直跟踪进程的峰值内存使用（写入到stats中），而且它可以在Scrapy进程使用的内存超过了限制的时候，中止掉进程，并且在这一切发生的时候使用email发出通知。<br>

##### 5、MEMUSAGE_LIMIT_MB
　　默认值：0<br>
　　作用域：scrapy.extensions.memusage<br>
　　Scrapy最大允许使用的内存，单位为MB，超过这个值，Scrapy进程将会被终止。如果值为0，那么将被不会进行检查。<br>

##### 6、MEMUSAGE_CHECK_INTERVAL_SECONDS
　　默认值：60.0<br>
　　作用域：scrapy.extensions.memusage<br>
　　每隔多长时间检查一次内存的使用量，单位为秒。<br>

##### 7、MEMUSAGE_NOTIFY_MAIL
　　默认值：False<br>
　　作用域：scrapy.extensions.memusage<br>
　　接收内存使用报告的邮箱地址。<br>

##### 8、MEMUSAGE_WARNING_MB
　　默认值：0<br>
　　作用域：scrapy.extensions.memusage<br>
　　使用内存超过这个值时，将会发出email警告，单位为MB。如果值为0，那么将被不会进行检查。<br>

#### 6、广度优先和深度优先
##### 1、DEPTH_LIMIT
　　默认值：0<br>
　　作用域：scrapy.spidermiddlewares.depth.DepthMiddleware<br>
　　对于任一网站，允许爬取的最大深度。如果值为0，则不会有任何限制。<br>

##### 2、DEPTH_PRIORITY
　　默认值：0<br>
　　作用域：scrapy.spidermiddlewares.depth.DepthMiddleware<br>
　　用来根据深度调整请求优先级的值。<br>
　　a、如果为0，则不会有任何优先级的调整。<br>
　　b、如果值为正数，那么会降低优先级，也就是说深度更深的请求将会排在爬取队列的后方。这也意味着深度较浅的请求将会优先爬取。这通常被用于广度优先爬取（BFO）。<br>
　　c、如果值为负数，那么会提高优先级，也就是说深度更深的请求会优先爬取。这通常被用于深度优先爬取（DFO）。<br>
##### 3、DEPTH_STATS
　　默认值：True<br>
　　作用域：scrapy.spidermiddlewares.depth.DepthMiddleware<br>
　　是否收集最大深度的统计数据。<br>

##### 4、DEPTH_STATS_VERBOSE
　　默认值：False<br>
　　作用域：scrapy.spidermiddlewares.depth.DepthMiddleware<br>
　　是否收集最大深度的详细统计数据。如果启用，那么每个深度请求的数量将会被收集到数据中。<br>

#### 7、重试和重定向　　
##### 1、REDIRECT_ENABLED
　　默认值：True<br>
　　是否启用重定向middleware。

##### 2、REDIRECT_MAX_TIMES
　　默认值：20<br>
　　请求的最大重定向次数。重定向的次数超过这个值后，相应将被原样返回。<br>

##### 3、REDIRECT_PRIORITY_ADJUST
　　默认值：+2<br>
　　作用域：scrapy.downloadermiddlewares.redirect.RedirectMiddleware<br>
　　用来调整重定向的请求跟原始请求的优先级：<br>
　　a、值为正数表示重定向的请求优先级更高。<br>
　　b、值为负数表示原始请求优先级更高。<br>

##### 4、RETRY_ENABLED
　　默认值：True<br>
　　是否启用重试middleware。<br>

##### 5、RETRY_TIMES
　　默认值：2<br>
　　除了第一次请求外，最大的重试次数。<br>
　　这个值也可以在Request.meta中的max_retry_times字段进行设置。max_retry_times的优先级比RETRY_TIMES的优先级更高。<br>

##### 6、RETRY_HTTP_CODES
　　默认值：[500, 502, 503, 504, 408]<br>
　　需要重试的响应状态码。<br>

##### 7、RETRY_PRIORITY_ADJUST
　　默认值：-1<br>
　　作用域：scrapy.downloadermiddlewares.retry.RetryMiddleware<br>
　　用来调整重试的请求跟原始请求的优先级：<br>
　　a、值为正数表示重试的请求优先级更高。<br>
　　b、值为负数表示原始请求优先级更高。<br>